### This is Owen Monroe's Homework Assignment # 3 for Text Mining IS 567
October 8, 2023

1. Train and evaluate a unigram-based baseline classifier

In [284]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


In [285]:
# Loading Data and Removing Empties

train_df = pd.read_csv('chatgpt_train.csv')
test_df = pd.read_csv('chatgpt_test.csv')

full_train_df = train_df.dropna()
full_test_df = test_df.dropna()

print(len(full_train_df['review'].to_list()))
print(len(full_test_df['review'].to_list()))

full_train_df


1829
458


,date,title,review,rating
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4
...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1
1830,5/19/2023 1:23,Goated app,Best app,5
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1


In [286]:
# Function to Convert Ratings to Three-Way Ratings

def change_five_to_three(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    elif rating in [4, 5]:
        return 'positive'
    else:
        return 'unknown'

In [287]:
# New Rating List for Train and Test Data

three_rate_train_list = full_train_df['rating'].apply(change_five_to_three).to_list()
three_rate_test_list = full_test_df['rating'].apply(change_five_to_three).to_list()


full_train_df['three rating'] = three_rate_train_list
full_test_df['three rating'] = three_rate_test_list


/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_55994/896341391.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_train_df['three rating'] = three_rate_train_list


In [288]:
full_train_df

,date,title,review,rating,three rating
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4,positive
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4,positive
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4,positive
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4,positive
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4,positive
...,...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1,negative
1830,5/19/2023 1:23,Goated app,Best app,5,positive
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1,negative
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1,negative


In [289]:
full_test_df

,date,title,review,rating,three rating
0,5/19/2023 6:09,error unsupported country,cant login,2,negative
1,5/19/2023 9:39,Hype junk,More harm than help.,1,negative
2,5/19/2023 4:12,your gpt 4 is fake,Fix it,1,negative
3,5/20/2023 3:01,Please impose IPadOS,We need IPadOS!!!,5,positive
4,5/19/2023 20:49,Amazing,Great,5,positive
...,...,...,...,...,...
453,5/19/2023 0:17,Andrew Justino Wilson 5/19/23,This has to be a beginning to something crazy ...,5,positive
454,5/18/2023 19:13,Superb AI,I?€?ve been using chat and have been a proud p...,5,positive
455,5/18/2023 18:27,Fantastic App with Room for Enhancements,The ChatGPT iOS app is an outstanding product....,5,positive
456,5/18/2023 17:17,"Awesome technology, deplorable tactics",Sam Altman?€?s blatant attempt at regulatory c...,2,negative


In [290]:
train_rev_text = full_train_df["review"]
test_rev_text = full_test_df["review"]

# Setting n-gram range to uni-grams
vectorizer = CountVectorizer(ngram_range = (1, 1))

# Creating training data representation
train_rev_uni_cv = vectorizer.fit_transform(train_rev_text.values.astype('U'))
print(train_rev_uni_cv.shape,"\n") 

# Creating test data representation
test_rev_uni_cv = vectorizer.transform(test_rev_text.values.astype('U'))
print(test_rev_uni_cv.shape,"\n") 

print(train_rev_uni_cv)

(1829, 5551) 

(458, 5551) 

  (0, 5224)	1
  (0, 4985)	9
  (0, 4932)	5
  (0, 3613)	1
  (0, 5282)	1
  (0, 3134)	1
  (0, 493)	2
  (0, 5253)	2
  (0, 776)	2
  (0, 3331)	3
  (0, 3165)	2
  (0, 5454)	1
  (0, 1302)	3
  (0, 2139)	1
  (0, 806)	1
  (0, 5421)	1
  (0, 2666)	6
  (0, 1426)	1
  (0, 4267)	3
  (0, 3883)	2
  (0, 3184)	1
  (0, 2660)	3
  (0, 3692)	2
  (0, 1352)	1
  (0, 4906)	16
  :	:
  (1824, 2949)	1
  (1824, 3593)	1
  (1824, 2644)	1
  (1824, 5301)	1
  (1825, 291)	1
  (1825, 527)	1
  (1826, 2666)	1
  (1826, 2456)	1
  (1826, 413)	1
  (1826, 3247)	1
  (1826, 5433)	1
  (1826, 1695)	1
  (1827, 2666)	1
  (1827, 474)	1
  (1827, 5464)	1
  (1827, 2619)	1
  (1827, 3372)	1
  (1827, 1976)	1
  (1827, 2807)	1
  (1827, 794)	1
  (1827, 195)	1
  (1827, 3699)	1
  (1827, 2328)	1
  (1828, 2666)	1
  (1828, 2135)	1


In [291]:
# Define true labels from train set

x_train_cv = train_rev_uni_cv
y_train_rev_series = full_train_df["three rating"]
x_test_cv = test_rev_uni_cv
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv, y_train_rev_series)

# Predict the labels for the test data

predictions = mnb_model.predict(x_test_cv)
predictions

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'negative', 'positive'

In [292]:


true_label_list = y_test_rev_series.to_list()
predict_label_list = list(predictions)

In [293]:
# Getting Evaluation Metrics for NB Unigrams 

true_label_list = y_test_rev_series.to_list()
predict_label_list = list(predictions)

acc_score = accuracy_score(true_label_list, predict_label_list)

prec_score_macro = precision_score(true_label_list, predict_label_list, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list, average='weighted')


print('Evaluation Reports - Unigrams NB Baseline')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score for unigrams = {prec_score_macro}')
print(f'micro precision score for unigrams = {prec_score_micro}')
print(f'weighted precision score for unigrams = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score for unigrams = {rec_score_macro}')
print(f'micro recall score for unigrams = {rec_score_micro}')
print(f'weighted recall score for unigrams = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score for unigrams = {f1_score_macro}')
print(f'micro F1 score for unigrams = {f1_score_micro}')
print(f'weighted F1 score for unigrams = {f1_score_weighted}')


Evaluation Reports - Unigrams NB Baseline

Accuracy Score: 0.7358078602620087

Precision Scores
macro precision score for unigrams = 0.5420844714259355
micro precision score for unigrams = 0.7358078602620087
weighted precision score for unigrams = 0.7128098567692405

Recall Scores
macro recall score for unigrams = 0.5007752841737103
micro recall score for unigrams = 0.7358078602620087
weighted recall score for unigrams = 0.7358078602620087

F1 Scores
macro F1 score for unigrams = 0.5116057233704292
micro F1 score for unigrams = 0.7358078602620087
weighted F1 score for unigrams = 0.7176894078769239


2. Remove Features with Low Variance

In [294]:
from sklearn.feature_selection import VarianceThreshold

# Removing Features with threshold below 0.001

selector = VarianceThreshold(threshold = 0.001)

X_train_features_filtered_var_thr_1 = selector.fit(train_rev_uni_cv).transform(train_rev_uni_cv)
print ("Train feature space before filtering: ", train_rev_uni_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr_1.shape)

X_test_features_filtered_var_thr_1 = selector.transform(test_rev_uni_cv)
print ("Test feature space before filtering: ", test_rev_uni_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr_1.shape)

print(X_test_features_filtered_var_thr_1)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 3054)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 3054)
  (0, 389)	1
  (0, 1619)	1
  (1, 1243)	1
  (1, 1267)	1
  (1, 1753)	1
  (1, 2682)	1
  (2, 1067)	1
  (2, 1478)	1
  (3, 1467)	1
  (3, 1789)	1
  (3, 2960)	1
  (4, 1204)	1
  (5, 164)	1
  (5, 1187)	1
  (5, 1473)	1
  (5, 2168)	1
  (5, 2706)	1
  (6, 1063)	1
  (7, 164)	1
  (7, 1853)	1
  (8, 1195)	2
  (8, 1473)	1
  (8, 1890)	1
  (8, 2706)	1
  (10, 443)	1
  :	:
  (455, 2912)	1
  (455, 2991)	1
  (455, 3003)	2
  (455, 3004)	1
  (455, 3028)	5
  (456, 62)	1
  (456, 107)	1
  (456, 128)	1
  (456, 214)	1
  (456, 222)	1
  (456, 223)	1
  (456, 281)	1
  (456, 317)	1
  (456, 443)	1
  (456, 772)	1
  (456, 1091)	1
  (456, 1139)	1
  (456, 1846)	1
  (456, 1872)	2
  (456, 2330)	1
  (456, 2483)	1
  (456, 2687)	1
  (456, 2738)	1
  (456, 2972)	1
  (456, 3003)	1


In [295]:
# Training an NB Model for the Feature Space with Low Variance Features Removed threshold below 0.001

# Define true labels from train set

x_train_cv_low1 = X_train_features_filtered_var_thr_1
y_train_rev_series = full_train_df["three rating"]
x_test_cv_low1 = X_test_features_filtered_var_thr_1
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_low1, y_train_rev_series)

# Predict the labels for the test data

predictions_low1 = mnb_model.predict(x_test_cv_low1)
predictions_low1

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'negative', 'negative', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'positive', 'neutral', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'negative', 'positive

In [296]:
# Evaluation Metrics for NB with Low Variance Features Removed, Threshold of 0.001

true_label_list = y_test_rev_series.to_list()
predict_label_list_low1 = list(predictions_low1)

acc_score = accuracy_score(true_label_list, predict_label_list_low1)

prec_score_macro = precision_score(true_label_list, predict_label_list_low1, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_low1, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_low1, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_low1, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_low1, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_low1, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_low1, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_low1, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_low1, average='weighted')


print('Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.001 ')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.001 

Accuracy Score: 0.7270742358078602

Precision Scores
macro precision score = 0.5286054212610907
micro precision score = 0.7270742358078602
weighted precision score = 0.7060217680304899

Recall Scores
macro recall score = 0.4974315878104861
micro recall score = 0.7270742358078602
weighted recall score = 0.7270742358078602

F1 Scores
macro F1 score = 0.5065362940263409
micro F1 score = 0.7270742358078602
weighted F1 score = 0.7117444835328599


In [297]:
selector_5 = VarianceThreshold(threshold = 0.005)

X_train_features_filtered_var_thr_5 = selector_5.fit(train_rev_uni_cv).transform(train_rev_uni_cv)
print ("Train feature space before filtering: ", train_rev_uni_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_var_thr_5.shape)

X_test_features_filtered_var_thr_5 = selector_5.transform(test_rev_uni_cv)
print ("Test feature space before filtering: ", test_rev_uni_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_var_thr_5.shape)

print(X_test_features_filtered_var_thr_5)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 980)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 980)
  (0, 507)	1
  (1, 387)	1
  (1, 546)	1
  (1, 831)	1
  (2, 320)	1
  (2, 459)	1
  (3, 558)	1
  (3, 932)	1
  (4, 367)	1
  (5, 65)	1
  (5, 361)	1
  (5, 455)	1
  (5, 680)	1
  (5, 846)	1
  (6, 318)	1
  (7, 65)	1
  (7, 577)	1
  (8, 364)	2
  (8, 455)	1
  (8, 589)	1
  (8, 846)	1
  (10, 142)	1
  (10, 572)	1
  (10, 579)	1
  (10, 914)	1
  :	:
  (455, 870)	1
  (455, 891)	1
  (455, 900)	3
  (455, 901)	4
  (455, 908)	1
  (455, 913)	1
  (455, 954)	2
  (455, 955)	1
  (455, 966)	5
  (456, 25)	1
  (456, 40)	1
  (456, 83)	1
  (456, 84)	1
  (456, 104)	1
  (456, 142)	1
  (456, 235)	1
  (456, 325)	1
  (456, 342)	1
  (456, 572)	1
  (456, 584)	2
  (456, 775)	1
  (456, 835)	1
  (456, 856)	1
  (456, 936)	1
  (456, 954)	1


In [298]:
# Training an NB Model for the Feature Space with Low Variance Features Removed threshold below 0.005

# Define true labels from train set

x_train_cv_low5 = X_train_features_filtered_var_thr_5
y_train_rev_series = full_train_df["three rating"]
x_test_cv_low5 = X_test_features_filtered_var_thr_5
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_low5, y_train_rev_series)

# Predict the labels for the test data

predictions_low5 = mnb_model.predict(x_test_cv_low5)
predictions_low5

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'neutral', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'neutral',
       'negative', 'neutral', 'positive', 'negative', 'positive',
       'negative', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'neutral', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive',

In [299]:
# Evaluation Metrics for NB with Low Variance Features Removed, Threshold of 0.005

true_label_list = y_test_rev_series.to_list()
predict_label_list_low5 = list(predictions_low5)

acc_score = accuracy_score(true_label_list, predict_label_list_low5)

prec_score_macro = precision_score(true_label_list, predict_label_list_low5, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_low5, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_low5, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_low5, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_low5, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_low5, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_low5, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_low5, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_low5, average='weighted')


print('Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.005 ')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB with Low Variance Features Removed, Threshold of 0.005 

Accuracy Score: 0.7358078602620087

Precision Scores
macro precision score = 0.5362426035502958
micro precision score = 0.7358078602620087
weighted precision score = 0.7181904214361385

Recall Scores
macro recall score = 0.4995527704912724
micro recall score = 0.7358078602620087
weighted recall score = 0.7358078602620087

F1 Scores
macro F1 score = 0.5100713456114153
micro F1 score = 0.7358078602620087
weighted F1 score = 0.7195576238803115


3. Select top k-best features using information gain 

In [300]:
# Selecting K-best features with k = 1000

from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=1000)
X_train_features_filtered_kbest1 = selector.fit_transform(x_train_cv, y_train_rev_series)
print ("Train feature space before filtering: ", x_train_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest1.shape)

X_test_features_filtered_kbest1 = selector.transform(x_test_cv)
print ("Test feature space before filtering: ", x_test_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest1.shape)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 1000)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 1000)


In [301]:
# Training an NB Model for the Feature Space with K-best features k = 1000

# Define true labels from train set

x_train_cv_k1 = X_train_features_filtered_kbest1
y_train_rev_series = full_train_df["three rating"]
x_test_cv_k1 = X_test_features_filtered_kbest1
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_k1, y_train_rev_series)

# Predict the labels for the test data

predictions_k1 = mnb_model.predict(x_test_cv_k1)
predictions_k1

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'negative',
       'negative', 'negative', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive'

In [302]:
# Evaluation Metrics for NB for the Feature Space with K-best features k = 1000

true_label_list = y_test_rev_series.to_list()
predict_label_list_k1 = list(predictions_k1)

acc_score = accuracy_score(true_label_list, predict_label_list_k1)

prec_score_macro = precision_score(true_label_list, predict_label_list_k1, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_k1, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_k1, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_k1, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_k1, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_k1, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_k1, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_k1, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_k1, average='weighted')


print('Evaluation Reports - NB for the Feature Space with K-best features k = 1000')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB wfor the Feature Space with K-best features k = 1000

Accuracy Score: 0.7489082969432315

Precision Scores
macro precision score = 0.5667439727843752
micro precision score = 0.7489082969432315
weighted precision score = 0.7331256698532228

Recall Scores
macro recall score = 0.5185938663816834
micro recall score = 0.7489082969432315
weighted recall score = 0.7489082969432315

F1 Scores
macro F1 score = 0.5327846831403961
micro F1 score = 0.7489082969432315
weighted F1 score = 0.7320941141075613


In [303]:
# Selecting K-best features with k = 2000

from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=2000)
X_train_features_filtered_kbest2 = selector.fit_transform(x_train_cv, y_train_rev_series)
print ("Train feature space before filtering: ", x_train_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest2.shape)

X_test_features_filtered_kbest2 = selector.transform(x_test_cv)
print ("Test feature space before filtering: ", x_test_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest2.shape)

Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 2000)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 2000)


In [304]:
# Training an NB Model for the Feature Space with K-best features k = 2000

# Define true labels from train set

x_train_cv_k2 = X_train_features_filtered_kbest2
y_train_rev_series = full_train_df["three rating"]
x_test_cv_k2 = X_test_features_filtered_kbest2
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_k2, y_train_rev_series)

# Predict the labels for the test data

predictions_k2 = mnb_model.predict(x_test_cv_k2)
predictions_k2

array(['negative', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'negative', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'neutral', 'neutral', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive'

In [305]:
# Evaluation Metrics for NB for the Feature Space with K-best features k = 2000

true_label_list = y_test_rev_series.to_list()
predict_label_list_k2 = list(predictions_k2)

acc_score = accuracy_score(true_label_list, predict_label_list_k2)

prec_score_macro = precision_score(true_label_list, predict_label_list_k2, average='macro')
prec_score_micro = precision_score(true_label_list, predict_label_list_k2, average='micro')
prec_score_weighted = precision_score(true_label_list, predict_label_list_k2, average='weighted')

rec_score_macro = recall_score(true_label_list, predict_label_list_k2, average='macro')
rec_score_micro = recall_score(true_label_list, predict_label_list_k2, average='micro')
rec_score_weighted = recall_score(true_label_list, predict_label_list_k2, average='weighted')

f1_score_macro = f1_score(true_label_list, predict_label_list_k2, average='macro')
f1_score_micro = f1_score(true_label_list, predict_label_list_k2, average='micro')
f1_score_weighted = f1_score(true_label_list, predict_label_list_k2, average='weighted')


print('Evaluation Reports - NB for the Feature Space with K-best features k = 2000')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB for the Feature Space with K-best features k = 2000

Accuracy Score: 0.7379912663755459

Precision Scores
macro precision score = 0.5207628839081421
micro precision score = 0.7379912663755459
weighted precision score = 0.7152427888953939

Recall Scores
macro recall score = 0.48728002849833213
micro recall score = 0.7379912663755459
weighted recall score = 0.7379912663755459

F1 Scores
macro F1 score = 0.49498356240232805
micro F1 score = 0.7379912663755459
weighted F1 score = 0.7179665704150677


4. Lexicon-based feature selection

In [306]:
# Creating Lexicon Word Lists

positive_words = open('positive_words_list.txt', 'r')
posi_word_str = positive_words.read()
posi_word_list = posi_word_str.replace('\n', ' ').split(' ')

negative_words = open('negative_words_list.txt', 'r')
neg_word_str = negative_words.read()
neg_word_list = neg_word_str.replace('\n', ' ').split(' ')

lex_word_list =  neg_word_list + posi_word_list
lex_word_list

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [307]:
# Filtering the DataFrame to only include lexicon words

# https://stackoverflow.com/questions/61926076/how-to-remove-words-from-a-data-frame-that-are-not-in-list-in-python

full_train_df['lex review'] = full_train_df.review.apply(lambda p: ' '.join(w for w in p.split() if w in lex_word_list))
full_train_df[['lex review', 'review', 'rating']][0:50]

full_test_df['lex review'] = full_test_df.review.apply(lambda p: ' '.join(w for w in p.split() if w in lex_word_list))

/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_55994/1885815054.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_train_df['lex review'] = full_train_df.review.apply(lambda p: ' '.join(w for w in p.split() if w in lex_word_list))


In [344]:
# Creating a New Model based on Lexicon Word list
# Building the Representation 

train_lex_text = full_train_df["lex review"]
test_lex_text = full_test_df["lex review"]

# Setting n-gram range to uni-grams
vectorizer = CountVectorizer(ngram_range = (1, 1))

# Creating training data representation
train_lex_cv = vectorizer.fit_transform(train_lex_text.values.astype('U'))
print(train_lex_cv.shape,"\n") 

# Creating test data representation
test_lex_cv = vectorizer.transform(test_lex_text.values.astype('U'))
print(test_lex_cv.shape,"\n") 

print(train_lex_cv)

(1829, 915) 

(458, 915) 

  (0, 609)	2
  (0, 183)	1
  (0, 12)	1
  (0, 341)	1
  (0, 892)	1
  (0, 105)	1
  (0, 29)	1
  (0, 736)	3
  (0, 782)	1
  (0, 347)	1
  (0, 56)	1
  (0, 493)	1
  (0, 5)	1
  (0, 219)	1
  (0, 287)	1
  (1, 736)	1
  (1, 782)	1
  (1, 493)	1
  (1, 219)	1
  (1, 266)	2
  (1, 7)	1
  (1, 605)	1
  (1, 559)	1
  (1, 508)	1
  (1, 356)	1
  :	:
  (1813, 54)	1
  (1817, 786)	1
  (1818, 273)	1
  (1818, 799)	2
  (1818, 278)	1
  (1818, 83)	1
  (1818, 47)	1
  (1818, 491)	1
  (1818, 898)	2
  (1818, 681)	1
  (1818, 798)	1
  (1818, 519)	2
  (1818, 533)	1
  (1818, 541)	1
  (1818, 66)	1
  (1818, 589)	1
  (1818, 650)	1
  (1818, 808)	1
  (1818, 231)	1
  (1818, 376)	1
  (1818, 904)	1
  (1818, 67)	1
  (1826, 39)	1
  (1827, 610)	1
  (1828, 341)	1


In [310]:
# Training a Model

# Define true labels from train set

x_train_cv_lex = train_lex_cv
y_train_rev_series = full_train_df["three rating"]
x_test_cv_lex = test_lex_cv
y_test_rev_series = full_test_df["three rating"]

# Build model on the training data

mnb_model = MultinomialNB()
mnb_model.fit(x_train_cv_lex, y_train_rev_series)

# Predict the labels for the test data

predictions_lex = mnb_model.predict(x_test_cv_lex)
predictions_lex


array(['positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'neutral', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positiv

In [311]:
true_label_list = y_test_rev_series.to_list()
predict_label_list_lex = list(predictions_lex)

acc_score = accuracy_score(true_label_list, predictions_lex)

prec_score_macro = precision_score(true_label_list, predictions_lex, average='macro')
prec_score_micro = precision_score(true_label_list, predictions_lex, average='micro')
prec_score_weighted = precision_score(true_label_list, predictions_lex, average='weighted')

rec_score_macro = recall_score(true_label_list, predictions_lex, average='macro')
rec_score_micro = recall_score(true_label_list, predictions_lex, average='micro')
rec_score_weighted = recall_score(true_label_list, predictions_lex, average='weighted')

f1_score_macro = f1_score(true_label_list, predictions_lex, average='macro')
f1_score_micro = f1_score(true_label_list, predictions_lex, average='micro')
f1_score_weighted = f1_score(true_label_list, predictions_lex, average='weighted')


print('Evaluation Reports - NB for the Feature Space with Lexicon based features')
print()
print(f'Accuracy Score: {acc_score}')
print()
print('Precision Scores')
print(f'macro precision score = {prec_score_macro}')
print(f'micro precision score = {prec_score_micro}')
print(f'weighted precision score = {prec_score_weighted}')
print()
print('Recall Scores')
print(f'macro recall score = {rec_score_macro}')
print(f'micro recall score = {rec_score_micro}')
print(f'weighted recall score = {rec_score_weighted}')
print()
print('F1 Scores')
print(f'macro F1 score = {f1_score_macro}')
print(f'micro F1 score = {f1_score_micro}')
print(f'weighted F1 score = {f1_score_weighted}')

Evaluation Reports - NB for the Feature Space with Lexicon based features

Accuracy Score: 0.6593886462882096

Precision Scores
macro precision score = 0.43833048833550886
micro precision score = 0.6593886462882096
weighted precision score = 0.6233608011186476

Recall Scores
macro recall score = 0.3630865636840572
micro recall score = 0.6593886462882096
weighted recall score = 0.6593886462882096

F1 Scores
macro F1 score = 0.324883655733109
micro F1 score = 0.6593886462882096
weighted F1 score = 0.5610832559010729


6. Extract and Select Best Unigrams 

In [312]:
# Selecting K-best features with k = 2000

from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=2000)
X_train_features_filtered_kbest2 = selector.fit_transform(x_train_cv, y_train_rev_series)
print ("Train feature space before filtering: ", x_train_cv.shape)
print ("Train feature space after filtering: ", X_train_features_filtered_kbest2.shape)

X_test_features_filtered_kbest2 = selector.transform(x_test_cv)
print ("Test feature space before filtering: ", x_test_cv.shape)
print ("Test feature space after filtering: ", X_test_features_filtered_kbest2.shape)
print(y_train_rev_series.shape)
print(y_test_rev_series.shape)


Train feature space before filtering:  (1829, 5551)
Train feature space after filtering:  (1829, 2000)
Test feature space before filtering:  (458, 5551)
Test feature space after filtering:  (458, 2000)
(1829,)
(458,)


7. Train and Evaluate a Naive Bayes classifier using cross-validation

In [313]:
y_train_array = y_train_rev_series.values
print(y_train_array.shape)

(1829,)


In [345]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Create a Multinomial Naive Bayes model
mnb_model = MultinomialNB()

skf = StratifiedKFold(n_splits=5)

f1_scores_nb = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_train_features_filtered_kbest2, y_train_array), 1):
    X_train = X_train_features_filtered_kbest2[train_index]
    X_test = X_train_features_filtered_kbest2[test_index]
    Y_train = y_train_array[train_index]
    Y_test = y_train_array[test_index]

    mnb_model.fit(X_train, Y_train)
    predictions_kfold = mnb_model.predict(X_test)
    
    acc_score = accuracy_score(Y_test, predictions_kfold)
    prec_score_weighted = precision_score(Y_test, predictions_kfold, average='weighted')
    rec_score_weighted = recall_score(Y_test, predictions_kfold, average='weighted')
    f1_score_weighted_nb = f1_score(Y_test, predictions_kfold, average='weighted')

    f1_scores_nb.append(f1_score_weighted_nb)

    print(f'Fold {fold_idx} for NB with kbest = 2000, 5 fold cross-validation:')
    print(f'Accuracy Score: {acc_score}')
    print(f'Weighted Precision Score: {prec_score_weighted}')
    print(f'Weighted Recall Score: {rec_score_weighted}')
    print(f'Weighted F1 Score: {f1_score_weighted_nb}')
    print()

print(f'Weighted F1 Score for NB Classifier: {f1_scores_nb}')
print()


Fold 1 for NB with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7814207650273224
Weighted Precision Score: 0.7546294236715083
Weighted Recall Score: 0.7814207650273224
Weighted F1 Score: 0.7523821717880836

Fold 2 for NB with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7622950819672131
Weighted Precision Score: 0.7578797527358261
Weighted Recall Score: 0.7622950819672131
Weighted F1 Score: 0.7488948197322874

Fold 3 for NB with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7759562841530054
Weighted Precision Score: 0.7693163436170125
Weighted Recall Score: 0.7759562841530054
Weighted F1 Score: 0.7650201919325562

Fold 4 for NB with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.8005464480874317
Weighted Precision Score: 0.7983973058695532
Weighted Recall Score: 0.8005464480874317
Weighted F1 Score: 0.7875139029933749

Fold 5 for NB with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7890410958904109
Weighted Precision Score: 0.738

8. Training a Linear SVM Classifier

In [346]:
# Building a Linear SVM Classifier using the k=2000 kbest features with 5-fold cross validation

from sklearn.svm import LinearSVC

svm = LinearSVC()

skf = StratifiedKFold(n_splits=5)

f1_scores_svm = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_train_features_filtered_kbest2, y_train_array), 1):
    X_train = X_train_features_filtered_kbest2[train_index]
    X_test = X_train_features_filtered_kbest2[test_index]
    Y_train = y_train_array[train_index]
    Y_test = y_train_array[test_index]

    svm.fit(X_train, Y_train)
    predictions_svm = svm.predict(X_test)
    
    acc_score = accuracy_score(Y_test, predictions_svm)
    prec_score_weighted = precision_score(Y_test, predictions_svm, average='weighted')
    rec_score_weighted = recall_score(Y_test, predictions_svm, average='weighted')
    f1_score_weighted_svm = f1_score(Y_test, predictions_svm, average='weighted')

    f1_scores_svm.append(f1_score_weighted_svm)

    print(f'Fold {fold_idx} for SVM with kbest = 2000, 5 fold cross-validation:')
    print(f'Accuracy Score: {acc_score}')
    print(f'Weighted Precision Score: {prec_score_weighted}')
    print(f'Weighted Recall Score: {rec_score_weighted}')
    print(f'Weighted F1 Score: {f1_score_weighted_svm}')
    print()


print(f'F1 Scores List for SVM Classifier: {f1_scores_svm}')
print()

Fold 1 SVM with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.680327868852459
Weighted Precision Score: 0.6722251593005485
Weighted Recall Score: 0.680327868852459
Weighted F1 Score: 0.6753358415487529

Fold 2 SVM with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7021857923497268
Weighted Precision Score: 0.7266452062939771
Weighted Recall Score: 0.7021857923497268
Weighted F1 Score: 0.7128095551776402

Fold 3 SVM with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.6857923497267759
Weighted Precision Score: 0.6716295431227868
Weighted Recall Score: 0.6857923497267759
Weighted F1 Score: 0.6777760249704226

Fold 4 SVM with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7540983606557377
Weighted Precision Score: 0.7350966747636427
Weighted Recall Score: 0.7540983606557377
Weighted F1 Score: 0.7397134339363093

Fold 5 SVM with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7205479452054795
Weighted Precision Score: 0.6867177900150185
Wei

9. Training a Logistic Regression Classifier

In [347]:
# Building a Logistic Regression Classifier using the k=2000 kbest features with 5-fold cross validation

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer

f1_scores_lr = []

logreg = LogisticRegression(solver="liblinear", penalty="l1")

skf = StratifiedKFold(n_splits=5)

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_train_features_filtered_kbest2, y_train_array), 1):
    X_train = X_train_features_filtered_kbest2[train_index]
    X_test = X_train_features_filtered_kbest2[test_index]
    Y_train = y_train_array[train_index]
    Y_test = y_train_array[test_index]

    logreg.fit(X_train, Y_train)
    predictions_lr = logreg.predict(X_test)
    
    acc_score = accuracy_score(Y_test, predictions_lr)
    prec_score_weighted = precision_score(Y_test, predictions_lr, average='weighted')
    rec_score_weighted = recall_score(Y_test, predictions_lr, average='weighted')
    f1_score_weighted_lr = f1_score(Y_test, predictions_lr, average='weighted')

    f1_scores_lr.append(f1_score_weighted_lr)


    print(f'Fold {fold_idx} for Logistic Regression Classifier with kbest = 2000, 5 fold cross-validation:')
    print(f'Accuracy Score: {acc_score}')
    print(f'Weighted Precision Score: {prec_score_weighted}')
    print(f'Weighted Recall Score: {rec_score_weighted}')
    print(f'Weighted F1 Score: {f1_score_weighted_lr}')
    print()

print(f' Weighted F1 Score for Logistic Regression: {f1_scores_lr}')
print()

Fold 1 for Logistic Regression Classifier with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.680327868852459
Weighted Precision Score: 0.6637314577244461
Weighted Recall Score: 0.680327868852459
Weighted F1 Score: 0.6711642843855948

Fold 2 for Logistic Regression Classifier with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.7021857923497268
Weighted Precision Score: 0.6880777591160104
Weighted Recall Score: 0.7021857923497268
Weighted F1 Score: 0.6941893311877188

Fold 3 for Logistic Regression Classifier with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.674863387978142
Weighted Precision Score: 0.6249423390816834
Weighted Recall Score: 0.674863387978142
Weighted F1 Score: 0.6458191809006956

Fold 4 for Logistic Regression Classifier with kbest = 2000, 5 fold cross-validation:
Accuracy Score: 0.726775956284153
Weighted Precision Score: 0.6960043927831167
Weighted Recall Score: 0.726775956284153
Weighted F1 Score: 0.6898253575549457

Fold 5 for Logisti

10. Model Comparison Using a t-test

In [327]:
from scipy import stats

# LinearSVC vs. Naive Bayes t-test

svc_nb_ttest = stats.ttest_ind(f1_scores_svm, f1_scores_nb,)
print("t-test result SVC and NB: ", svc_nb_ttest) 

# LinearSVC vs Logistic Regression

svc_lr_ttest = stats.ttest_ind(f1_scores_svm, f1_scores_lr,)
print("t-test result SVC and LR: ", svc_lr_ttest) 

# Naive Bayes vs Logistic Regression

nb_lr_ttest = stats.ttest_ind(f1_scores_nb, f1_scores_lr,)
print("t-test result NB and LR: ", nb_lr_ttest)

t-test result SVC and NB:  Ttest_indResult(statistic=-4.4466151416194934, pvalue=0.002148950144364985)
t-test result SVC and LR:  Ttest_indResult(statistic=1.5358455037809184, pvalue=0.16312705181395892)
t-test result SVC and LR:  Ttest_indResult(statistic=7.725791715151462, pvalue=5.608100793024348e-05)


11. Hyperparameter Tuning

In [337]:
from sklearn.model_selection import GridSearchCV

from sklearn.svm import LinearSVC

svm = LinearSVC()

param_grid = {'C': [0.01, 0.1, 1, 10, 100, 1000]}

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, Y_train)

best_C = grid_search.best_params_['C']
best_accuracy = grid_search.best_score_

print(f"Best C value: {best_C}")
print(f"Best Accuracy: {best_accuracy}")


/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/Users/owenmonroe/anaconda3/envs/tex

Best C value: 0.1
Best Accuracy: 0.7288325774931039


In [340]:
# Best C Value = 0.1 with Accuracy = Best Accuracy: 0.7288325774931039
# Running the Linear SVC Model with that C Value

from sklearn.svm import LinearSVC

svm = LinearSVC(C=0.1)

skf = StratifiedKFold(n_splits=5)

f1_scores_svm = []

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_train_features_filtered_kbest2, y_train_array), 1):
    X_train = X_train_features_filtered_kbest2[train_index]
    X_test = X_train_features_filtered_kbest2[test_index]
    Y_train = y_train_array[train_index]
    Y_test = y_train_array[test_index]

    svm.fit(X_train, Y_train)
    predictions_svm = svm.predict(X_test)
    
    acc_score = accuracy_score(Y_test, predictions_svm)
    prec_score_weighted = precision_score(Y_test, predictions_svm, average='weighted')
    rec_score_weighted = recall_score(Y_test, predictions_svm, average='weighted')
    f1_score_weighted_svm = f1_score(Y_test, predictions_svm, average='weighted')

    f1_scores_svm.append(f1_score_weighted_svm)

    print(f'Fold {fold_idx} for SVM Classifier with C = 0.1:')
    print(f'Accuracy Score: {acc_score}')
    print(f'Weighted Precision Score: {prec_score_weighted}')
    print(f'Weighted Recall Score: {rec_score_weighted}')
    print(f'Weighted F1 Score: {f1_score_weighted_svm}')
    print()


print(f'F1 Scores List for SVM Classifier: {f1_scores_svm}')
print()

Fold 1 for SVM Classifier with C = 0.1:
Accuracy Score: 0.6885245901639344
Weighted Precision Score: 0.6827053973732145
Weighted Recall Score: 0.6885245901639344
Weighted F1 Score: 0.6848756576532293

Fold 2 for SVM Classifier with C = 0.1:
Accuracy Score: 0.726775956284153
Weighted Precision Score: 0.7209931899592448
Weighted Recall Score: 0.726775956284153
Weighted F1 Score: 0.7234081828931319

Fold 3 for SVM Classifier with C = 0.1:
Accuracy Score: 0.7076502732240437
Weighted Precision Score: 0.6543492936935559
Weighted Recall Score: 0.7076502732240437
Weighted F1 Score: 0.676255096138491

Fold 4 for SVM Classifier with C = 0.1:
Accuracy Score: 0.7650273224043715
Weighted Precision Score: 0.7588775335882374
Weighted Recall Score: 0.7650273224043715
Weighted F1 Score: 0.735960533371943

Fold 5 for SVM Classifier with C = 0.1:
Accuracy Score: 0.6958904109589041
Weighted Precision Score: 0.63343870741131
Weighted Recall Score: 0.6958904109589041
Weighted F1 Score: 0.6229003186637857

F

In [348]:
# Logistic Regression with C=0.1

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer

f1_scores_lr = []

logreg = LogisticRegression(solver="liblinear", penalty="l1", C=0.1)

skf = StratifiedKFold(n_splits=5)

for fold_idx, (train_index, test_index) in enumerate(skf.split(X_train_features_filtered_kbest2, y_train_array), 1):
    X_train = X_train_features_filtered_kbest2[train_index]
    X_test = X_train_features_filtered_kbest2[test_index]
    Y_train = y_train_array[train_index]
    Y_test = y_train_array[test_index]

    logreg.fit(X_train, Y_train)
    predictions_lr = logreg.predict(X_test)
    
    acc_score = accuracy_score(Y_test, predictions_lr)
    prec_score_weighted = precision_score(Y_test, predictions_lr, average='weighted')
    rec_score_weighted = recall_score(Y_test, predictions_lr, average='weighted')
    f1_score_weighted_lr = f1_score(Y_test, predictions_lr, average='weighted')

    f1_scores_lr.append(f1_score_weighted_lr)


    print(f'Fold {fold_idx} for Logistic Regression Classifier with C=0.1:')
    print(f'Accuracy Score: {acc_score}')
    print(f'Weighted Precision Score: {prec_score_weighted}')
    print(f'Weighted Recall Score: {rec_score_weighted}')
    print(f'Weighted F1 Score: {f1_score_weighted_lr}')
    print()

print(f' Weighted F1 Score for Logistic Regression: {f1_scores_lr}')
print()

Fold 1 for Logistic Regression Classifier with C=0.1:
Accuracy Score: 0.6939890710382514
Weighted Precision Score: 0.6190646718658814
Weighted Recall Score: 0.6939890710382514
Weighted F1 Score: 0.6345482985131958

Fold 2 for Logistic Regression Classifier with C=0.1:
Accuracy Score: 0.6939890710382514
Weighted Precision Score: 0.6132043861045261
Weighted Recall Score: 0.6939890710382514
Weighted F1 Score: 0.6325667726274002

Fold 3 for Logistic Regression Classifier with C=0.1:
Accuracy Score: 0.6775956284153005
Weighted Precision Score: 0.618549518762485
Weighted Recall Score: 0.6775956284153005
Weighted F1 Score: 0.6007652327540162

Fold 4 for Logistic Regression Classifier with C=0.1:
Accuracy Score: 0.6557377049180327
Weighted Precision Score: 0.5882123341139734
Weighted Recall Score: 0.6557377049180327
Weighted F1 Score: 0.5698634285358923

Fold 5 for Logistic Regression Classifier with C=0.1:
Accuracy Score: 0.6657534246575343
Weighted Precision Score: 0.6004755821218423
Weighte

/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/owenmonroe/anaconda3/envs/textmining/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/